In [1]:
import pandas as pd
import re
from time import sleep
from selenium.webdriver.common.by import By
from selenium import webdriver
import numpy as np

In [2]:
# import the data set
df = pd.read_csv("Test_file.csv")


In [3]:

class TimeTravel:
    
    def __init__(self, destination: str, source: str):
        """
        Initialize the TimeTravel object with a destination, source, and Selenium WebDriver.
        """
        self.destination = destination
        self.source = source
        self.driver = webdriver.Chrome()
        self.__url = ""
        # self.driver.minimize_window()  # Corrected: self.driver.minimize_window()
        
    def car_button_click(self):
        """
        Constructs the Google Maps URL, opens it in the driver, and clicks the car button to set route mode to driving.
        """
        # Construct URL for Google Maps directions from source to destination
        base_url = "https://www.google.com/maps/dir/"
        self.__url = f"{base_url}{self.source}/{self.destination}/"

        # Open the URL in the driver
        self.driver.get(self.__url)
        print(f"Navigating to URL: {self.__url}")
        sleep(10)  # Wait for the page to load
        
        # Click on the car button to set driving mode
        car_button = self.driver.find_element(By.XPATH, 
            '/html/body/div[2]/div[3]/div[8]/div[3]/div[1]/div[2]/div/div[2]/div/div/div/div[2]/button')
            
        car_button.click()
        sleep(5)
        # Update the URL after selecting the driving mode
        self.__url = self.driver.current_url
        print(f"Updated URL with driving mode: {self.__url}")
        
    def add_time_traveling(self, time_stamp: int):
        """
        Modifies the URL to include a specific travel time based on a timestamp (in milliseconds).
        Retrieves the estimated travel time from the web page.
        """
        # Add timestamp to the current URL
        if not self.__url:
            raise ValueError("URL is not set. Please call car_button_click() first.")
        
        current_url = self.__url
        add_8_index = re.search(r"4m1", current_url)
        
        if add_8_index:
            current_url = current_url[:add_8_index.end()] + "8" + current_url[add_8_index.end()+1:]
        
        add_7_index = re.search(r"8!4m1", current_url)
        if add_7_index:
            current_url = current_url[:add_7_index.end()] + "7" + current_url[add_7_index.end()+1:]
        
        print(current_url)
        #test the new url as it is changes 
        add_in_between = re.search(r"!3e0?", current_url)
        if add_in_between:
            current_url = current_url[:add_in_between.start()]
        print("-----------------")
        print(current_url)

        unknown_part = "!2m3!6e0!7e2!8j"
        timestamp_str = str(int(time_stamp / 1000))  # Convert to seconds
        car_code = "!3e0?"
        new_url = current_url + unknown_part + timestamp_str + car_code
        
        print(f"Modified URL with timestamp: {new_url}")
        self.driver.get(new_url)

        # Fetch the estimated travel time
        sleep(5)  # Allow some time for the page to update
        # try:
        print()
        distance_element = self.driver.find_element(By.XPATH, 
            "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[1]/div[1]/div/div[1]/div[1]/span")
        travel_time = distance_element.text
        return travel_time


https://www.google.com/maps/dir/Haymarket+Square,+Boston,+MA,+USA/Northeastern+University,+360+Huntington+Ave,+Boston,+MA+02115,+United+States/@42.3538328,-71.0963806,14z/data=!3m1!4b1!4m18!4m17!1m5!1m1!1s0x89e3708f060b5dc7:0x7d0014b91aacbdee!2m2!1d-71.0585053!2d42.3637994!1m5!1m1!1s0x89e37a1999cf5ce1:0xc97b00e66522b98c!2m2!1d-71.0898892!2d42.339904!2m3!6e0!7e2!8j1730200500!3e0?entry=ttu&g_ep=EgoyMDI0MTAyOS4wIKXMDSoASAFQAw%3D%3D

In [4]:
completed_list = []
# crate the list of destination and source
df['traveling_time'] = np.nan
list_of_destination = list(df['destination'].unique())
list_of_source = list(df['source'].unique())
     

In [5]:
sum = 1

# as the destination ahe source is same for mank time 
# the only thing changes is timestamp so after press of car button we are going to change only time stamp and store the data inside the csv
# from selenium import webdriver
# tik = time.time()

for source in list_of_source:
  for destination in list_of_destination:
    cheak = TimeTravel(destination , source)
    cheak.car_button_click()
    for i in range(len(df)):
      # print(type(df['traveling_time'][i]) == float)
      # if type(df['traveling_time'][i]) == float:
      if df['destination'][i] == destination and df['source'][i] == source:
        print(f"time stamp is {df['time_stamp'][i]} and index is {i} ")
        df['traveling_time'][i] = cheak.add_time_traveling(df['time_stamp'][i])

    completed_list.append((source , destination))
    del cheak
    print(f"source = {source} and destination = {destination}")


Navigating to URL: https://www.google.com/maps/dir/Haymarket Square/North Station/
Updated URL with driving mode: https://www.google.com/maps/dir/Haymarket+Square,+Boston,+MA,+USA/North+Station,+135+Causeway+St,+Boston,+MA+02114,+United+States/@42.3650543,-71.062814,17z/data=!3m1!4b1!4m13!4m12!1m5!1m1!1s0x89e3708f060b5dc7:0x7d0014b91aacbdee!2m2!1d-71.0585053!2d42.3637994!1m5!1m1!1s0x89e3708e162d05cb:0x651087057818603a!2m2!1d-71.061974!2d42.3664424?entry=ttu&g_ep=EgoyMDI1MDIwNS4xIKXMDSoASAFQAw%3D%3D
time stamp is 1544950000000.0 and index is 0 
https://www.google.com/maps/dir/Haymarket+Square,+Boston,+MA,+USA/North+Station,+135+Causeway+St,+Boston,+MA+02114,+United+States/@42.3650543,-71.062814,17z/data=!3m1!4b1!4m18!4m17!1m5!1m1!1s0x89e3708f060b5dc7:0x7d0014b91aacbdee!2m2!1d-71.0585053!2d42.3637994!1m5!1m1!1s0x89e3708e162d05cb:0x651087057818603a!2m2!1d-71.061974!2d42.3664424?entry=ttu&g_ep=EgoyMDI1MDIwNS4xIKXMDSoASAFQAw%3D%3D
-----------------
https://www.google.com/maps/dir/Haymarket+

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[1]/div[1]/div/div[1]/div[1]/span"}
  (Session info: chrome=132.0.6834.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010468b0d4 cxxbridge1$str$ptr + 2600792
1   chromedriver                        0x00000001046839f0 cxxbridge1$str$ptr + 2570356
2   chromedriver                        0x00000001042243d8 cxxbridge1$string$len + 89376
3   chromedriver                        0x0000000104269414 cxxbridge1$string$len + 372060
4   chromedriver                        0x00000001042a3638 cxxbridge1$string$len + 610176
5   chromedriver                        0x000000010425d34c cxxbridge1$string$len + 322708
6   chromedriver                   

In [ ]:
df

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,traveling_time
0,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,5.0,1,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,typically 4 min
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,typically 3 min
2,0.44,Lyft,1.543370e+12,North Station,Haymarket Square,7.0,1,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,typically 3 min
3,0.44,Lyft,1.543550e+12,North Station,Haymarket Square,26.0,1,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,typically 3 min
4,0.44,Lyft,1.543460e+12,North Station,Haymarket Square,9.0,1,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,typically 3 min
5,0.44,Lyft,1.545070e+12,North Station,Haymarket Square,16.5,1,f6f6d7e4-3e18-4922-a5f5-181cdd3fa6f2,lyft_lux,Lux Black,typically 4 min
6,1.08,Lyft,1.543210e+12,Northeastern University,Back Bay,10.5,1,462816a3-820d-408b-8549-0b39e82f65ac,lyft_plus,Lyft XL,typically 3–6 min
7,1.08,Lyft,1.543780e+12,Northeastern University,Back Bay,16.5,1,474d6376-bc59-4ec9-bf57-4e6d6faeb165,lyft_lux,Lux Black,typically 4–10 min
8,1.08,Lyft,1.543820e+12,Northeastern University,Back Bay,3.0,1,4f9fee41-fde3-4767-bbf1-a00e108701fb,lyft_line,Shared,typically 4–8 min
9,1.08,Lyft,1.543320e+12,Northeastern University,Back Bay,27.5,1,8612d909-98b8-4454-a093-30bd48de0cb3,lyft_luxsuv,Lux Black XL,typically 5–12 min
